In [1]:
import hail as hl 
import pandas as pd 

hl.init()

Running on Apache Spark version 3.1.2
SparkUI available at http://192-168-1-12.tpgi.com.au:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.73-c6f6f09cec08
LOGGING: writing to /Users/elston.dsouza/Projects/UTR-Visualisation-App/pipeline/hail-20210816-1917-0.2.73-c6f6f09cec08.log


In [3]:
gnomad = hl.read_table("gs://gcp-public-data--gnomad/release/3.1.1/ht/genomes/gnomad.genomes.v3.1.1.sites.ht")


In [17]:
utr_regions = hl.Table.from_pandas(pd.read_csv("UTR_regions.tsv", sep="\t", names = ["CHROM", "POS", "POS_TO"]))

# Parse intervals
utr_regions = utr_regions.annotate(interval = hl.parse_locus_interval(hl.format("chr%s:%s-%s", utr_regions.CHROM,
                                                                utr_regions.POS,
                                                                utr_regions.POS_TO), 
                                                                     reference_genome='GRCh38'))
utr_regions = utr_regions.key_by("interval")

In [26]:
utr_regions.count()

27206

2021-08-16 19:31:12 Hail: INFO: Ordering unsorted dataset with network shuffle


In [28]:
gnomad = gnomad.filter(hl.is_defined(utr_regions[gnomad.locus]))


# Run VEP alongside UTR Annotator 
gnomad.show()

# Filter to consequence on MANE Transcript



ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54993)
Traceback (most recent call last):
  File "/Users/elston.dsouza/Environments/miniconda3/envs/utr_app_pipeline/lib/python3.7/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/elston.dsouza/Environments/miniconda3/envs/utr_app_pipeline/lib/python3.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54993)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:54993)
Traceback (most recent call last):
  File "/Users/elston.dsouza/Environments/miniconda3/envs/utr_app_pipeline/lib/python3.7/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/elston.dsouza/Environments/miniconda3/envs/utr_app_pipeline/lib/python3.7/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:54993)